### 1. Spark RDD operations

In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/02 21:14:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

```
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    SUM(total_amount) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
````

In [3]:
df_green.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

In [4]:
df_green.rdd

MapPartitionsRDD[11] at javaToPython at NativeMethodAccessorImpl.java:0

In [5]:
df_green.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), lpep_dropoff_datetime=datetime.datetime(2020, 1, 20, 15, 46), store_and_fwd_flag=None, RatecodeID=None, PULocationID=67, DOLocationID=39, passenger_count=None, trip_distance=8.0, fare_amount=29.9, extra=2.75, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=33.45, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41)

In [6]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [8]:
rdd.filter(lambda row: False).take(1)

[]

In [9]:
from datetime import datetime

In [10]:
start = datetime(year=2020, month=1, day=1)

rdd.filter(lambda row: row.lpep_pickup_datetime >= start).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)]

In [11]:
def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [12]:
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)]

In [13]:
rows = rdd.take(10)
row = rows[0]
row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

#### group by 

In [15]:
def prepare_for_grouping(row): 
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount, count)

    return (key, value)

In [16]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .take(10)

[((datetime.datetime(2020, 1, 23, 13, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 67), (33.45, 1)),
 ((datetime.datetime(2020, 1, 15, 20, 0), 260), (8.3, 1)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (12.74, 1)),
 ((datetime.datetime(2020, 1, 15, 11, 0), 179), (5.8, 1)),
 ((datetime.datetime(2020, 1, 16, 8, 0), 41), (25.05, 1)),
 ((datetime.datetime(2020, 1, 28, 17, 0), 75), (21.33, 1)),
 ((datetime.datetime(2020, 1, 22, 14, 0), 152), (7.8, 1)),
 ((datetime.datetime(2020, 1, 31, 10, 0), 75), (26.05, 1))]

In [17]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)

In [18]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(10)

[((datetime.datetime(2020, 1, 20, 15, 0), 67), (79.5, 3)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (500.4700000000002, 33)),
 ((datetime.datetime(2020, 1, 16, 8, 0), 41), (736.1399999999996, 54)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 75), (608.9999999999999, 47)),
 ((datetime.datetime(2020, 1, 17, 21, 0), 74), (594.87, 39)),
 ((datetime.datetime(2020, 1, 3, 9, 0), 61), (142.21, 9)),
 ((datetime.datetime(2020, 1, 13, 19, 0), 223), (213.83000000000004, 19)),
 ((datetime.datetime(2020, 1, 16, 13, 0), 14), (163.73999999999998, 6)),
 ((datetime.datetime(2020, 1, 12, 15, 0), 82), (265.7900000000001, 29)),
 ((datetime.datetime(2020, 1, 8, 16, 0), 72), (130.67, 7))]

In [19]:
from collections import namedtuple

In [20]:
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])

In [21]:
def unwrap(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1],
        revenue=row[1][0],
        count=row[1][1]
    )

In [22]:
from pyspark.sql import types

In [23]:
result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True),
    types.StructField('zone', types.IntegerType(), True),
    types.StructField('revenue', types.DoubleType(), True),
    types.StructField('count', types.IntegerType(), True)
])

In [24]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) \
    .show()

+-------------------+----+------------------+-----+
|               hour|zone|           revenue|count|
+-------------------+----+------------------+-----+
|2020-01-20 15:00:00|  67|              79.5|    3|
|2020-01-05 16:00:00|  82| 500.4700000000002|   33|
|2020-01-16 08:00:00|  41| 736.1399999999996|   54|
|2020-01-20 15:00:00|  75| 608.9999999999999|   47|
|2020-01-17 21:00:00|  74|            594.87|   39|
|2020-01-03 09:00:00|  61|            142.21|    9|
|2020-01-13 19:00:00| 223|213.83000000000004|   19|
|2020-01-16 13:00:00|  14|163.73999999999998|    6|
|2020-01-12 15:00:00|  82| 265.7900000000001|   29|
|2020-01-08 16:00:00|  72|            130.67|    7|
|2020-01-03 11:00:00|  51| 305.9599999999999|   10|
|2020-01-30 09:00:00|  22|114.80000000000001|    4|
|2020-01-31 18:00:00| 260| 439.3100000000002|   27|
|2020-01-10 20:00:00|  66|            405.88|   21|
|2020-01-31 21:00:00|  41| 588.1600000000001|   40|
|2020-01-23 22:00:00| 166|            297.84|   18|
|2020-01-30 

In [26]:
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(unwrap) \
    .toDF(result_schema) 

In [27]:
df_result.write.parquet('tmp/green-revenue')

In [28]:
!ls tmp/green-revenue

_SUCCESS
part-00000-29a1bb01-3b87-4b9f-afd5-f85dee20bb69-c000.snappy.parquet
part-00001-29a1bb01-3b87-4b9f-afd5-f85dee20bb69-c000.snappy.parquet
part-00002-29a1bb01-3b87-4b9f-afd5-f85dee20bb69-c000.snappy.parquet
part-00003-29a1bb01-3b87-4b9f-afd5-f85dee20bb69-c000.snappy.parquet


### 2. Spark RDD mapPartition

In [29]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

df_green \
    .select(columns) \
    .show()

+--------+--------------------+------------+------------+-------------+
|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|
+--------+--------------------+------------+------------+-------------+
|       2| 2020-01-23 13:10:15|          74|         130|        12.77|
|    null| 2020-01-20 15:09:00|          67|          39|          8.0|
|       2| 2020-01-15 20:23:41|         260|         157|         1.27|
|       2| 2020-01-05 16:32:26|          82|          83|         1.25|
|       2| 2020-01-29 19:22:42|         166|          42|         1.84|
|       2| 2020-01-15 11:07:42|         179|         223|         0.76|
|       2| 2020-01-16 08:22:29|          41|         237|         3.32|
|       2| 2020-01-28 17:05:28|          75|         161|         2.21|
|       1| 2020-01-22 14:51:37|         152|         166|          0.9|
|       2| 2020-01-31 10:25:04|          75|         234|          6.1|
|       2| 2020-01-20 15:50:54|          75|          41|       

In [30]:
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [31]:
duration_rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [32]:
def apply_model_in_batch(partition):
    return [1]

In [33]:
rdd.mapPartitions(apply_model_in_batch).collect()
#collect() basically convert [[1],[1],[1],[1]] to [1, 1, 1, 1]

[1, 1, 1, 1]

In [34]:
def apply_model_in_batch(partition):
    cnt = 0
    for row in partition:
        cnt = cnt + 1
    return [cnt]

rdd.mapPartitions(apply_model_in_batch).collect()

[1141148, 438057, 432402, 292910]

In [35]:
import pandas as pd

In [36]:
rows = duration_rdd.take(10)

In [37]:
df = pd.DataFrame(rows, columns=columns)

In [38]:
df

,VendorID,lpep_pickup_datetime,PULocationID,DOLocationID,trip_distance
0,2.0,2020-01-23 13:10:15,74,130,12.77
1,NaN,2020-01-20 15:09:00,67,39,8.00
2,2.0,2020-01-15 20:23:41,260,157,1.27
3,2.0,2020-01-05 16:32:26,82,83,1.25
4,2.0,2020-01-29 19:22:42,166,42,1.84
5,2.0,2020-01-15 11:07:42,179,223,0.76
6,2.0,2020-01-16 08:22:29,41,237,3.32
7,2.0,2020-01-28 17:05:28,75,161,2.21
8,1.0,2020-01-22 14:51:37,152,166,0.90
9,2.0,2020-01-31 10:25:04,75,234,6.10


In [39]:
#model = ...

def model_predict(df):
#     y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [40]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] = predictions

    for row in df.itertuples():
        yield row

In [41]:
duration_rdd.mapPartitions(apply_model_in_batch).take(10)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, predicted_duration=63.849999999999994),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, predicted_duration=40.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, predicted_duration=6.35),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, predicted_duration=6.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, predicted_duration=9.200000000000001),
 Pandas(Index=5, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 11:07:42'), PULocationID=179, DOLocationID=223, trip_distance=0.7

In [42]:
df_predicts = duration_rdd \
    .mapPartitions(apply_model_in_batch)\
    .toDF() \
    .drop('Index')

In [43]:
df_predicts.select(['trip_distance','predicted_duration']).show()

+-------------+------------------+
|trip_distance|predicted_duration|
+-------------+------------------+
|        12.77|63.849999999999994|
|          8.0|              40.0|
|         1.27|              6.35|
|         1.25|              6.25|
|         1.84| 9.200000000000001|
|         0.76|               3.8|
|         3.32|16.599999999999998|
|         2.21|             11.05|
|          0.9|               4.5|
|          6.1|              30.5|
|         1.74|               8.7|
|         1.18|5.8999999999999995|
|          2.2|              11.0|
|         3.04|              15.2|
|         0.85|              4.25|
|         5.06|25.299999999999997|
|         1.57|7.8500000000000005|
|          6.8|              34.0|
|         1.06| 5.300000000000001|
|         1.23|              6.15|
+-------------+------------------+
only showing top 20 rows

